In [14]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPoint, MultiPolygon
from shapely import affinity
from shapely.ops import unary_union, voronoi_diagram
import math
import rasterio
from rasterstats import zonal_stats
import pandas as pd
import matplotlib.pyplot as plt


In [15]:
input_file = "data/Foundation_Data.gpkg"
processing_file = "outputs/coastal_protection_processing_layers.gpkg"
output_file = "outputs/Jamaica_coastal_protection_areas.gpkg"
raster_file = "data/flood_rasters/JamaicaJAM001RCPbaseline2010_epsg_32618_RP_100.tif"

coastline_edge_layer = gpd.read_file(input_file, layer="edges")
coastline_node_layer = gpd.read_file(input_file, layer="nodes")

coastline_edge_layer.to_file(output_file, layer="edges", driver="GPKG")
coastline_node_layer.to_file(output_file, layer="nodes", driver="GPKG")

buffer_distance_km = 2
node_offset = 0

In [16]:
def create_bounding_boxes_for_edges(edge_layer, node_layer):
    """
    Create a bounding box around each edge using its start and end nodes,
    center it on the midpoint of the diagonal line, and apply rotation.

    Parameters:
    - edge_layer (GeoDataFrame): GeoDataFrame containing edges (LineString) with from_id and to_id attributes
    - node_layer (GeoDataFrame): GeoDataFrame containing nodes (Point) with id attributes

    Returns:
    - GeoDataFrame: A GeoDataFrame with rotated and translated bounding box polygons around each edge
    """
    bounding_boxes = []

    # Lists to store distances, midpoints, and angles
    distances = []
    midpoints = []
    angles = []

    for _, edge in edge_layer.iterrows():
        from_node_id = edge['from_id']
        to_node_id = edge['to_id']
        
        # Get the node geometries (points) based on 'from_id' and 'to_id' IDs
        from_node = node_layer[node_layer['node_id'] == from_node_id].geometry.iloc[0]
        to_node = node_layer[node_layer['node_id'] == to_node_id].geometry.iloc[0]

        if (round(from_node.x,5) == round(to_node.x,5)) and (round(from_node.y,5) == round(to_node.y,5)):
            # print(to_node.x, to_node.y ,"-", from_node.x, from_node.y)
            continue

        # Calculate the straight-line distance between the two nodes (this is the diagonal)
        distance = from_node.distance(to_node)
        distances.append(distance)

        # Calculate the midpoint of the diagonal line between the nodes
        midpoint = calculate_midpoint(from_node, to_node)
        midpoints.append(midpoint)

        # Calculate the angle of rotation (between the line and the horizontal axis)
        angle = calculate_angle(from_node, to_node)
        angles.append(angle)

        # Calculate the bounding box's vertical span (height)
        min_y = min(from_node.y, to_node.y)
        max_y = max(from_node.y, to_node.y)
        height = max_y - min_y + (buffer_distance_km * 1000)

        # Use the calculated distance as the width of the bounding box
        min_x = min(from_node.x, to_node.x)
        max_x = min_x + distance - node_offset # Set the width to the calculated distance

        # Create the initial bounding box polygon (not yet translated or rotated)
        bounding_box = Polygon([
            (min_x, min_y),
            (min_x, min_y + height),  # Ensure consistent height
            (max_x, min_y + height),  # Ensure consistent height
            (max_x, min_y),  # Close the rectangle with the calculated width
            (min_x, min_y)  # Closing the rectangle
        ])

        # Translate the bounding box to center it on the midpoint of the diagonal line
        translated_bounding_box = translate_bounding_box(bounding_box, midpoint)

        # Rotate the bounding box to align with the diagonal line
        rotated_bounding_box = rotate_bounding_box(translated_bounding_box, midpoint, angle)

        bounding_boxes.append(rotated_bounding_box)

    # Convert list of rotated and translated bounding boxes into a GeoDataFrame
    bounding_boxes_gdf = gpd.GeoDataFrame(geometry=bounding_boxes, crs=edge_layer.crs)
    bounding_boxes_gdf['id'] = range(len(bounding_boxes_gdf))  # Assign unique ids for each bounding box
    bounding_boxes_gdf['distance'] = distances  # Add the distance column
    # bounding_boxes_gdf['midpoint'] = midpoints  # Add the midpoint column
    bounding_boxes_gdf['angle'] = angles  # Add the angle column

    return bounding_boxes_gdf

def calculate_midpoint(from_node, to_node):
    """
    Calculate the midpoint between two nodes.

    Parameters:
    - from_node (Point): The start node as a Shapely Point
    - to_node (Point): The end node as a Shapely Point

    Returns:
    - Point: The midpoint between the two nodes as a Shapely Point
    """
    # Calculate the midpoint coordinates
    midpoint_x = (from_node.x + to_node.x) / 2
    midpoint_y = (from_node.y + to_node.y) / 2
    return Point(midpoint_x, midpoint_y)

def calculate_angle(from_node, to_node):
    """
    Calculate the angle between the line connecting two nodes and the horizontal axis.

    Parameters:
    - from_node (Point): The start node as a Shapely Point
    - to_node (Point): The end node as a Shapely Point

    Returns:
    - float: The angle in degrees between the line and the horizontal axis
    """
    # Calculate the angle in radians between the two points
    delta_x = to_node.x - from_node.x
    delta_y = to_node.y - from_node.y
    angle_rad = math.atan2(delta_y, delta_x)
    
    # Convert the angle from radians to degrees
    angle_deg = math.degrees(angle_rad)
    return angle_deg

def translate_bounding_box(bounding_box, midpoint):
    """
    Translate a bounding box so that it is centered on a specific midpoint.

    Parameters:
    - bounding_box (Polygon): The bounding box polygon to translate
    - midpoint (Point): The target center point for translation

    Returns:
    - Polygon: The translated bounding box
    """
    # Calculate the current center of the bounding box
    current_center_x = (bounding_box.bounds[0] + bounding_box.bounds[2]) / 2
    current_center_y = (bounding_box.bounds[1] + bounding_box.bounds[3]) / 2

    # Calculate the translation vector (difference between current center and midpoint)
    translate_x = midpoint.x - current_center_x
    translate_y = midpoint.y - current_center_y

    # Translate the bounding box by the translation vector
    return affinity.translate(bounding_box, xoff=translate_x, yoff=translate_y)

def rotate_bounding_box(bounding_box, midpoint, angle):
    """
    Rotate the bounding box around its center by a given angle.

    Parameters:
    - bounding_box (Polygon): The bounding box polygon to rotate
    - midpoint (Point): The center point around which the bounding box will rotate
    - angle (float): The angle in degrees by which to rotate the bounding box

    Returns:
    - Polygon: The rotated bounding box
    """
    # Rotate the bounding box by the angle around the midpoint
    return affinity.rotate(bounding_box, angle, origin=(midpoint.x, midpoint.y))

# Create bounding boxes around edges, calculate distance, midpoint, angle, and apply rotation
bounding_boxes_gdf = create_bounding_boxes_for_edges(coastline_edge_layer, coastline_node_layer)

bounding_boxes_gdf.to_file(processing_file, layer='rectangles', driver='GPKG')


In [17]:
def order_edges_around_island(edge_layer):
    """
    Order edges around an island, starting with edge_0 and following the
    connections based on from_id and to_id.

    Parameters:
    - edge_layer (GeoDataFrame): GeoDataFrame containing edges with 'id', 'from_id', and 'to_id' columns

    Returns:
    - List[str]: List of edge IDs in the order they are traced around the island
    """
    # Initialize the ordered list with the first edge
    ordered_edges = []

    # Create a lookup dictionary for edges by their 'from_id'
    edges_by_from_id = {
        edge['from_id']: edge
        for _, edge in edge_layer.iterrows()
    }

    # Start with edge_0
    current_edge = edge_layer[edge_layer['id'] == 'edge_0'].iloc[0]
    ordered_edges.append(current_edge['rectangle_id'])

    # Track the starting node for circular detection
    start_node = current_edge['from_id']

    # Follow the edges until we complete the loop
    while True:
        # Get the next edge based on the current edge's to_id
        next_from_id = current_edge['to_id']

        # Break the loop if we return to the starting edge
        if next_from_id == start_node:
            break

        # Find the next edge using the lookup dictionary
        current_edge = edges_by_from_id[next_from_id]
        ordered_edges.append(current_edge['rectangle_id'])

    return ordered_edges

ordered_edge_ids = order_edges_around_island(coastline_edge_layer)
print(ordered_edge_ids)

[np.int64(0), 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [18]:
def add_max_zonal_stats(geopkg, raster_path, new_column_name):
    """
    Adds a new column to a GeoDataFrame with the maximum raster value for each polygon,
    and reprojects it back to its original CRS after the operation.
    """
    # Save the original CRS of the GeoDataFrame
    original_crs = geopkg.crs
    
    # Open the raster to get its CRS
    with rasterio.open(raster_path) as src:
        raster_crs = src.crs

    # Reproject GeoDataFrame to match raster CRS
    if not geopkg.crs == raster_crs:
        geopkg = geopkg.to_crs(raster_crs)
    
    # Ensure geometries are valid
    geopkg["geometry"] = geopkg["geometry"].buffer(0)

    # Perform zonal statistics to find the maximum raster value within each polygon
    stats = zonal_stats(
        geopkg,
        raster_path,
        stats="max",
        geojson_out=False,
        nodata=-9999  # Adjust this to match your raster's NoData value if applicable
    )
    
    # Extract the max values and assign them to the new column
    geopkg[new_column_name] = [stat['max'] if stat['max'] is not None else None for stat in stats]
    
    # Reproject back to the original CRS
    if not geopkg.crs == original_crs:
        geopkg = geopkg.to_crs(original_crs)
    
    return geopkg

# Example usage
gdf = gpd.read_file(processing_file, layer="rectangles")
new_column_name = "max_flood_height"

# Process and add max stats
updated_gdf = add_max_zonal_stats(gdf, raster_file, new_column_name)

# Save the updated GeoDataFrame to the file, ensuring CRS consistency
updated_gdf.to_file(processing_file, layer="rectangles", driver="GPKG")


In [19]:
def join_polygons_by_flood_height(gdf, flood_height_threshold, area_limit, group_size):

    # Initialize variables for processing
    groups = []
    heights = []  # To store maximum flood heights for each group
    ids = []  # To store original IDs for each group
    current_group = []
    current_heights = []
    current_ids = []
    is_below_threshold = None

    for index, row in gdf.iterrows():
        if row["max_flood_height"] > flood_height_threshold:
            if is_below_threshold is True or (current_group and unary_union(current_group).area > area_limit):
                # End the below-threshold group or large area group
                if len(current_group) == group_size and groups:
                    # Add the single polygon to the previous group
                    groups[-1] = unary_union([groups[-1], current_group[0]]).convex_hull
                    heights[-1] = max(heights[-1], current_heights[0])
                    ids[-1].extend(current_ids)
                else:
                    # Finalize the group normally
                    combined_geometry = unary_union(current_group)
                    groups.append(combined_geometry.convex_hull)
                    heights.append(max(current_heights))
                    ids.append(current_ids)

                current_group = []
                current_heights = []
                current_ids = []

            # Switch to above-threshold mode
            is_below_threshold = False
            current_group.append(row["geometry"])
            current_heights.append(row["max_flood_height"])
            current_ids.append(row["id"])

        else:  # Polygons with height below or equal to the threshold
            if is_below_threshold is False or (current_group and unary_union(current_group).area > area_limit):
                # End the above-threshold group or large area group
                if len(current_group) == group_size and groups:
                    # Add the single polygon to the previous group
                    groups[-1] = unary_union([groups[-1], current_group[0]]).convex_hull
                    heights[-1] = max(heights[-1], current_heights[0])
                    ids[-1].extend(current_ids)
                else:
                    # Finalize the group normally
                    combined_geometry = unary_union(current_group)
                    groups.append(combined_geometry.convex_hull)
                    heights.append(max(current_heights))
                    ids.append(current_ids)

                current_group = []
                current_heights = []
                current_ids = []

            # Switch to below-threshold mode
            is_below_threshold = True
            current_group.append(row["geometry"])
            current_heights.append(row["max_flood_height"])
            current_ids.append(row["id"])

    # Finalize the last group if any polygons remain
    if current_group:
        if len(current_group) == group_size and groups:
            # Add the single polygon to the previous group
            groups[-1] = unary_union([groups[-1], current_group[0]]).convex_hull
            heights[-1] = max(heights[-1], current_heights[0])
            ids[-1].extend(current_ids)
        else:
            # Finalize the group normally
            combined_geometry = unary_union(current_group)
            groups.append(combined_geometry.convex_hull)
            heights.append(max(current_heights))
            ids.append(current_ids)

    # Convert the list of grouped geometries, heights, and IDs back to a GeoDataFrame
    result_gdf = gpd.GeoDataFrame(
        {
            "geometry": gpd.GeoSeries(groups),
            "max_flood_height": heights,
            "original_ids": ids,  # Store the list of original IDs
        }
    )

    # Add an ID field to the result GeoDataFrame
    result_gdf["id"] = range(len(result_gdf))

    # Set CRS
    result_gdf.set_crs(gdf.crs, inplace=True)

    return result_gdf

# Load your polygon layer
gdf = gpd.read_file(processing_file, layer="rectangles")
ordered_edge_ids = [eid for eid in ordered_edge_ids if eid in gdf["id"].values]
input_polygons = gdf.set_index("id").loc[ordered_edge_ids].reset_index()

# Set the flood height threshold and area limit
flood_height_threshold = 0
area_limit = 10_000_000  
group_size = 1

joined_polygons = join_polygons_by_flood_height(input_polygons,flood_height_threshold, area_limit, group_size)

# Save the results to a GeoPackage
joined_polygons.to_file(processing_file, layer="joined_polygons", driver="GPKG")


In [20]:
def merge_overlapping_polygons(gdf, overlap_threshold):
    """
    Iteratively merges overlapping polygons that overlap by more than the specified threshold.
    
    Args:
        gdf: GeoDataFrame with 'geometry' and 'max_flood_height' fields.
        overlap_threshold: The minimum percentage of overlap to trigger merging.

    Returns:
        A new GeoDataFrame with polygons merged until no pair overlaps by more than the threshold.
    """
    
    # Ensure CRS is properly defined for the input data
    if gdf.crs is None:
        gdf.set_crs("EPSG:3097", inplace=True)  # Set CRS to Jamaica Metric Grid, adjust if necessary

    # Helper function to check and merge overlapping polygons
    def merge_once(gdf, overlap_threshold):
        """
        Performs one pass over the GeoDataFrame to merge overlapping polygons.
        """
        merged = []
        indices_to_merge = set()

        for i, geom1 in enumerate(gdf.geometry):
            if i in indices_to_merge:
                continue  # Skip already merged geometries

            for j, geom2 in enumerate(gdf.geometry):
                if i >= j or j in indices_to_merge:
                    continue  # Avoid redundant checks and already-merged geometries

                # Check intersection
                if geom1.intersects(geom2):
                    intersection = geom1.intersection(geom2)
                    if intersection.is_empty:
                        continue

                    # Check if the overlap is greater than the threshold
                    area1 = geom1.area
                    area2 = geom2.area
                    intersection_area = intersection.area

                    if (intersection_area / min(area1, area2)) > overlap_threshold:
                        # Merge using the convex hull
                        new_geom = unary_union([geom1, geom2]).convex_hull
                        new_height = max(gdf.loc[i, "max_flood_height"], gdf.loc[j, "max_flood_height"])

                        # Mark for merge
                        merged.append({
                            "geometry": new_geom,
                            "max_flood_height": new_height
                        })
                        indices_to_merge.update([i, j])

                        break  # Stop checking for the current geometry once merged

        # Retain non-overlapping geometries
        non_merged = gdf.loc[~gdf.index.isin(indices_to_merge)]

        # Combine the merged geometries with non-overlapping ones
        merged_gdf = gpd.GeoDataFrame(
            {
                "geometry": [m["geometry"] for m in merged],
                "max_flood_height": [m["max_flood_height"] for m in merged],
            }
        )

        # Ensure CRS is explicitly set for the merged geometries
        merged_gdf.set_crs(gdf.crs, inplace=True)

        # Combine with non-overlapping ones
        final_gdf = gpd.GeoDataFrame(pd.concat([non_merged, merged_gdf], ignore_index=True))

        # Set CRS again after concatenation
        final_gdf.set_crs(gdf.crs, inplace=True)

        return final_gdf

    # Iteratively resolve all overlaps until convergence
    prev_gdf = gdf
    while True:
        new_gdf = merge_once(prev_gdf, overlap_threshold)
        if len(new_gdf) == len(prev_gdf):  # No changes (no more overlaps)
            break
        prev_gdf = new_gdf

    # Ensure final CRS consistency
    prev_gdf.set_crs(gdf.crs, inplace=True)
    return prev_gdf

# if gdf.crs is None:
#     gdf.set_crs("EPSG:3097", inplace=True)  # Adjust the EPSG code to match your CRS if different

# Run the iterative merge function
overlap_threshold = 0.35
input_polygons = gpd.read_file(processing_file, layer="joined_polygons")
result_gdf = merge_overlapping_polygons(input_polygons, overlap_threshold)

# Reassign unique IDs starting from 1
result_gdf = result_gdf.reset_index(drop=True)  # Reset the index
result_gdf['id'] = result_gdf.index + 1        # Assign new unique IDs

# Save the results
result_gdf.to_file(processing_file, layer="merged_polygons",  driver="GPKG")



In [21]:
def create_trimmed_convex_hull(polygons):
    """
    Create a convex hull encompassing given polygons, 
    and compute the convex hull from their union.
    """
    # Combine the polygons and compute the convex hull
    combined = unary_union(polygons)
    convex_hull = combined.convex_hull

    return convex_hull

def connect_polygons_with_convex_hull(flood_polygons, ids):
    """
    Generate a trimmed convex hull connecting selected polygons 
    and save the result to a new GeoPackage. If no IDs are provided,
    all polygons in the layer will be used.
    """
    # Load GeoPackage
    gdf = flood_polygons

    # Determine which polygons to use based on provided IDs or all if the array is empty
    if ids:
        # Filter only the polygons by their IDs
        selected_gdf = gdf[gdf['id'].isin(ids)]
    else:
        # Use all polygons if no IDs are provided
        selected_gdf = gdf

    # Ensure geometry column exists and convert to list of geometries
    polygons = selected_gdf.geometry.tolist()

    # Create trimmed convex hull
    trimmed_hull = create_trimmed_convex_hull(polygons)

    # Prepare GeoDataFrame for the new polygon
    new_row = gpd.GeoDataFrame({
        'id': ['trimmed_hull'],
        'geometry': [trimmed_hull]
    }, crs=gdf.crs)


    return new_row


flood_polygons = gpd.read_file(processing_file, layer="merged_polygons")

# Case 2: No IDs, use all polygons
polygon_ids = []
jamaica_convex_hull = connect_polygons_with_convex_hull(flood_polygons, polygon_ids)

# Save the resulting polygon to a new GeoPackage
jamaica_convex_hull.to_file(processing_file, layer='jamaica_convex', driver='GPKG')


In [22]:
def create_clipped_voronoi(jamaica_polygon, coastline_polygons, polygon_ids):

    # Load the larger polygon
    convex_gdf = jamaica_polygon
    larger_polygon = convex_gdf.iloc[0].geometry  # Assuming the first row contains the larger polygon

    # Read the GeoPackage layer
    gdf = coastline_polygons

    # Filter polygons by the specified IDs (if provided)
    if polygon_ids:
        smaller_polygons = gdf[gdf['id'].isin(polygon_ids)].copy()
    else:
        # Use all polygons if no IDs are provided
        smaller_polygons = gdf

    # Ensure the CRS is consistent
    smaller_polygons.crs = gdf.crs

    # Extract centroids of smaller polygons
    centroids = smaller_polygons.geometry.centroid

    # Create a MultiPoint object from the centroids
    seed_points = MultiPoint([Point(c.x, c.y) for c in centroids])

    # Perform Voronoi tessellation using the centroids
    voronoi = voronoi_diagram(seed_points)

    # Clip Voronoi regions to the larger polygon
    clipped_regions = [region.intersection(larger_polygon) for region in voronoi.geoms]

    # Filter valid polygons from the clipped Voronoi regions
    valid_regions = [region for region in clipped_regions if not region.is_empty and isinstance(region, Polygon)]

    # Create a GeoDataFrame for the valid regions
    clipped_gdf = gpd.GeoDataFrame(
        {'id': range(len(valid_regions))},  # Assign a unique ID to each region
        geometry=valid_regions,
        crs=smaller_polygons.crs  # Use the same CRS as the smaller polygons
    )

    return clipped_gdf, centroids
    # print(f"Clipped Voronoi regions saved to {output_gpkg_path} in layer '{output_layer_name}'")

# Example of calling the function

jamaica_polygon = gpd.read_file(processing_file, layer = "jamaica_convex")
coastline_polygons = gpd.read_file(processing_file, layer = "merged_polygons")
polygon_ids = [] 

voronoi_polygons, centroid_points = create_clipped_voronoi(jamaica_polygon, coastline_polygons, polygon_ids)

voronoi_polygons.to_file(processing_file, layer="voronoi_polygons", driver="GPKG")
centroid_points.to_file(processing_file, layer="voronoi_points", driver="GPKG")


In [ ]:
#Function adopted from https://github.com/thomas-fred/jam-coastal-protection

def buffer_linestring_intersect_polygons(buffer_radius_km: float, linestring: gpd.GeoDataFrame, polygons: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Buffer a coastline by a given radius, then find the intersection with a set
    of polygons.

    Args:
        buffer_radius_km: The radius in km to buffer the coastline (how far you
            want the polygons to extend inland).
        linestring: A single-row GeoDataFrame with a linestring to buffer.
        polygons: A GeoDataFrame of polygons.

    Returns:
        A GeoDataFrame of polygons that have been clipped to the
        buffered linestring.
    """

    buffer = linestring.copy()
    buffer.geometry = linestring.geometry.buffer(buffer_radius_km * 1_000)

    cut_polygon_geoseries = buffer.geometry.iloc[0].intersection(polygons)

    cut_polygons = gpd.GeoDataFrame({"geometry": cut_polygon_geoseries})
    cut_polygons.crs = buffer.crs
    return cut_polygons


coast = coastline_edge_layer
buffer_radius_km = 1.5

coastal_polygons = gpd.read_file(processing_file, layer="voronoi_polygons")
coastal_polygons = coastal_polygons.geometry

coast_linestring = gpd.GeoDataFrame({"id": [0], "geometry":[coast.geometry.union_all()]})
coast_linestring.crs = coast.crs

buffered_voronoi = buffer_linestring_intersect_polygons(buffer_radius_km, coast_linestring, coastal_polygons)

buffered_voronoi.to_file(processing_file, layer="clipped_coastal_areas", driver="GPKG")


In [24]:
def clean_MultiPolygons(gdf):
    def largest_polygon(geom):
        if isinstance(geom, MultiPolygon):
            # If it's a MultiPolygon, find the largest one based on area
            return max(geom.geoms, key=lambda poly: poly.area)  # Access the 'geoms' attribute
        elif isinstance(geom, Polygon):
            # If it's a Polygon, return it as is
            return geom
        else:
            # If it's neither a Polygon nor a MultiPolygon, you can return it as is or handle differently
            return geom

    # Apply the function to the geometry column
    gdf['geometry'] = gdf['geometry'].apply(largest_polygon)
    return gdf

In [25]:
gdf = gpd.read_file(processing_file, layer="clipped_coastal_areas")

# Apply the function to keep the largest polygon for each MultiPolygon
cleaned_coastal_protection_area = clean_MultiPolygons(gdf)

# Now you can proceed with your processing and analysis
raster_path = raster_file
new_column_name = "max_flood_height"

# Process and add max stats
final_coastal_protection_area = add_max_zonal_stats(cleaned_coastal_protection_area, raster_path, new_column_name)

# Save the updated GeoDataFrame to the file, ensuring CRS consistency
final_coastal_protection_area.to_file(processing_file, layer="final_protection_area" ,driver="GPKG")
final_coastal_protection_area.to_file(output_file,driver="GPKG")
